In [65]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
from functools import reduce
from sklearn.preprocessing import MinMaxScaler

In [66]:
pd.set_option('display.max_columns', 1000)

In [67]:
# Defining directories
dir_path = Path.cwd().parent
data_folder = dir_path.joinpath("data/processed")

#training frames
s1_path = data_folder.joinpath("sent1_train_season_medians_widened.parquet")
s2_path = data_folder.joinpath("sent2_train_season_medians_widened.parquet")
cov_path = data_folder.joinpath("cov_final.parquet")

model_df_path = data_folder.joinpath("model_df.csv")
species_model_df_path = data_folder.joinpath('species_train_df.csv')
species_dict_path = dir_path.joinpath("data/species_dict.json")

index_cols = ["long", "lat", "species_names"]

In [68]:
s1 = pd.read_parquet(s1_path)
s1

,long,lat,species_names,vv_Descendingsummer2018,vv_Descendingsummer2019,vv_Descendingsummer2020,vv_Descendingsummer2021,vv_Descendingsummer2022,vv_Descendingsummer2023,vv_Descendingwinter2018,vv_Descendingwinter2019,vv_Descendingwinter2020,vv_Descendingwinter2021,vv_Descendingwinter2022,vv_Descendingwinter2023,vh_Descendingsummer2018,vh_Descendingsummer2019,vh_Descendingsummer2020,vh_Descendingsummer2021,vh_Descendingsummer2022,vh_Descendingsummer2023,vh_Descendingwinter2018,vh_Descendingwinter2019,vh_Descendingwinter2020,vh_Descendingwinter2021,vh_Descendingwinter2022,vh_Descendingwinter2023,vv_Ascendingsummer2018,vv_Ascendingsummer2019,vv_Ascendingsummer2020,vv_Ascendingsummer2021,vv_Ascendingsummer2022,vv_Ascendingsummer2023,vv_Ascendingwinter2018,vv_Ascendingwinter2019,vv_Ascendingwinter2020,vv_Ascendingwinter2021,vv_Ascendingwinter2022,vv_Ascendingwinter2023,vh_Ascendingsummer2018,vh_Ascendingsummer2019,vh_Ascendingsummer2020,vh_Ascendingsummer2021,vh_Ascendingsummer2022,vh_Ascendingsummer2023,vh_Ascendingwinter2018,vh_Ascendingwinter2019,vh_Ascendingwinter2020,vh_Ascendingwinter2021,vh_Ascendingwinter2022,vh_Ascendingwinter2023,VV_VH_Asummer2018,VV_VH_Asummer2019,VV_VH_Asummer2020,VV_VH_Asummer2021,VV_VH_Asummer2022,VV_VH_Asummer2023,VV_VH_Awinter2018,VV_VH_Awinter2019,VV_VH_Awinter2020,VV_VH_Awinter2021,VV_VH_Awinter2022,VV_VH_Awinter2023,VV_VH_Dsummer2018,VV_VH_Dsummer2019,VV_VH_Dsummer2020,VV_VH_Dsummer2021,VV_VH_Dsummer2022,VV_VH_Dsummer2023,VV_VH_Dwinter2018,VV_VH_Dwinter2019,VV_VH_Dwinter2020,VV_VH_Dwinter2021,VV_VH_Dwinter2022,VV_VH_Dwinter2023,VH_VV_Asummer2018,VH_VV_Asummer2019,VH_VV_Asummer2020,VH_VV_Asummer2021,VH_VV_Asummer2022,VH_VV_Asummer2023,VH_VV_Awinter2018,VH_VV_Awinter2019,VH_VV_Awinter2020,VH_VV_Awinter2021,VH_VV_Awinter2022,VH_VV_Awinter2023,VH_VV_Dsummer2018,VH_VV_Dsummer2019,VH_VV_Dsummer2020,VH_VV_Dsummer2021,VH_VV_Dsummer2022,VH_VV_Dsummer2023,VH_VV_Dwinter2018,VH_VV_Dwinter2019,VH_VV_Dwinter2020,VH_VV_Dwinter2021,VH_VV_Dwinter2022,VH_VV_Dwinter2023,SAR_NDVI_Asummer2018,SAR_NDVI_Asummer2019,SAR_NDVI_Asummer2020,SAR_NDVI_Asummer2021,SAR_NDVI_Asummer2022,SAR_NDVI_Asummer2023,SAR_NDVI_Awinter2018,SAR_NDVI_Awinter2019,SAR_NDVI_Awinter2020,SAR_NDVI_Awinter2021,SAR_NDVI_Awinter2022,SAR_NDVI_Awinter2023,SAR_NDVI_Dsummer2018,SAR_NDVI_Dsummer2019,SAR_NDVI_Dsummer2020,SAR_NDVI_Dsummer2021,SAR_NDVI_Dsummer2022,SAR_NDVI_Dsummer2023,SAR_NDVI_Dwinter2018,SAR_NDVI_Dwinter2019,SAR_NDVI_Dwinter2020,SAR_NDVI_Dwinter2021,SAR_NDVI_Dwinter2022,SAR_NDVI_Dwinter2023,DVI_Asummer2018,DVI_Asummer2019,DVI_Asummer2020,DVI_Asummer2021,DVI_Asummer2022,DVI_Asummer2023,DVI_Awinter2018,DVI_Awinter2019,DVI_Awinter2020,DVI_Awinter2021,DVI_Awinter2022,DVI_Awinter2023,DVI_Dsummer2018,DVI_Dsummer2019,DVI_Dsummer2020,DVI_Dsummer2021,DVI_Dsummer2022,DVI_Dsummer2023,DVI_Dwinter2018,DVI_Dwinter2019,DVI_Dwinter2020,DVI_Dwinter2021,DVI_Dwinter2022,DVI_Dwinter2023,SVI_Asummer2018,SVI_Asummer2019,SVI_Asummer2020,SVI_Asummer2021,SVI_Asummer2022,SVI_Asummer2023,SVI_Awinter2018,SVI_Awinter2019,SVI_Awinter2020,SVI_Awinter2021,SVI_Awinter2022,SVI_Awinter2023,SVI_Dsummer2018,SVI_Dsummer2019,SVI_Dsummer2020,SVI_Dsummer2021,SVI_Dsummer2022,SVI_Dsummer2023,SVI_Dwinter2018,SVI_Dwinter2019,SVI_Dwinter2020,SVI_Dwinter2021,SVI_Dwinter2022,SVI_Dwinter2023,RDVI_Asummer2018,RDVI_Asummer2019,RDVI_Asummer2020,RDVI_Asummer2021,RDVI_Asummer2022,RDVI_Asummer2023,RDVI_Awinter2018,RDVI_Awinter2019,RDVI_Awinter2020,RDVI_Awinter2021,RDVI_Awinter2022,RDVI_Awinter2023,RDVI_Dsummer2018,RDVI_Dsummer2019,RDVI_Dsummer2020,RDVI_Dsummer2021,RDVI_Dsummer2022,RDVI_Dsummer2023,RDVI_Dwinter2018,RDVI_Dwinter2019,RDVI_Dwinter2020,RDVI_Dwinter2021,RDVI_Dwinter2022,RDVI_Dwinter2023,NRDVI_Asummer2018,NRDVI_Asummer2019,NRDVI_Asummer2020,NRDVI_Asummer2021,NRDVI_Asummer2022,NRDVI_Asummer2023,NRDVI_Awinter2018,NRDVI_Awinter2019,NRDVI_Awinter2020,NRDVI_Awinter2021,NRDVI_Awinter2022,NRDVI_Awinter2023,NRDVI_Dsummer2018,NRDVI_Dsummer2019,NRDVI_Dsummer2020,NRDVI_Dsummer2021,NRDVI_Dsummer202

In [69]:
s2 = pd.read_parquet(s2_path)
s2

,long,lat,species_names,B11summer2018,B11summer2019,B11summer2020,B11summer2021,B11summer2022,B11summer2023,B11winter2018,B11winter2019,B11winter2020,B11winter2021,B11winter2022,B11winter2023,B02summer2018,B02summer2019,B02summer2020,B02summer2021,B02summer2022,B02summer2023,B02winter2018,B02winter2019,B02winter2020,B02winter2021,B02winter2022,B02winter2023,B12summer2018,B12summer2019,B12summer2020,B12summer2021,B12summer2022,B12summer2023,B12winter2018,B12winter2019,B12winter2020,B12winter2021,B12winter2022,B12winter2023,B08summer2018,B08summer2019,B08summer2020,B08summer2021,B08summer2022,B08summer2023,B08winter2018,B08winter2019,B08winter2020,B08winter2021,B08winter2022,B08winter2023,B04summer2018,B04summer2019,B04summer2020,B04summer2021,B04summer2022,B04summer2023,B04winter2018,B04winter2019,B04winter2020,B04winter2021,B04winter2022,B04winter2023,B03summer2018,B03summer2019,B03summer2020,B03summer2021,B03summer2022,B03summer2023,B03winter2018,B03winter2019,B03winter2020,B03winter2021,B03winter2022,B03winter2023,NDVIsummer2018,NDVIsummer2019,NDVIsummer2020,NDVIsummer2021,NDVIsummer2022,NDVIsummer2023,NDVIwinter2018,NDVIwinter2019,NDVIwinter2020,NDVIwinter2021,NDVIwinter2022,NDVIwinter2023,ARVI2summer2018,ARVI2summer2019,ARVI2summer2020,ARVI2summer2021,ARVI2summer2022,ARVI2summer2023,ARVI2winter2018,ARVI2winter2019,ARVI2winter2020,ARVI2winter2021,ARVI2winter2022,ARVI2winter2023,BWDRVIsummer2018,BWDRVIsummer2019,BWDRVIsummer2020,BWDRVIsummer2021,BWDRVIsummer2022,BWDRVIsummer2023,BWDRVIwinter2018,BWDRVIwinter2019,BWDRVIwinter2020,BWDRVIwinter2021,BWDRVIwinter2022,BWDRVIwinter2023,CVIsummer2018,CVIsummer2019,CVIsummer2020,CVIsummer2021,CVIsummer2022,CVIsummer2023,CVIwinter2018,CVIwinter2019,CVIwinter2020,CVIwinter2021,CVIwinter2022,CVIwinter2023,CTVIsummer2018,CTVIsummer2019,CTVIsummer2020,CTVIsummer2021,CTVIsummer2022,CTVIsummer2023,CTVIwinter2018,CTVIwinter2019,CTVIwinter2020,CTVIwinter2021,CTVIwinter2022,CTVIwinter2023,EVI2summer2018,EVI2summer2019,EVI2summer2020,EVI2summer2021,EVI2summer2022,EVI2summer2023,EVI2winter2018,EVI2winter2019,EVI2winter2020,EVI2winter2021,EVI2winter2022,EVI2winter2023,GVMIsummer2018,GVMIsummer2019,GVMIsummer2020,GVMIsummer2021,GVMIsummer2022,GVMIsummer2023,GVMIwinter2018,GVMIwinter2019,GVMIwinter2020,GVMIwinter2021,GVMIwinter2022,GVMIwinter2023,MSVAIhypersummer2018,MSVAIhypersummer2019,MSVAIhypersummer2020,MSVAIhypersummer2021,MSVAIhypersummer2022,MSVAIhypersummer2023,MSVAIhyperwinter2018,MSVAIhyperwinter2019,MSVAIhyperwinter2020,MSVAIhyperwinter2021,MSVAIhyperwinter2022,MSVAIhyperwinter2023,MTVI2summer2018,MTVI2summer2019,MTVI2summer2020,MTVI2summer2021,MTVI2summer2022,MTVI2summer2023,MTVI2winter2018,MTVI2winter2019,MTVI2winter2020,MTVI2winter2021,MTVI2winter2022,MTVI2winter2023,MNDVIsummer2018,MNDVIsummer2019,MNDVIsummer2020,MNDVIsummer2021,MNDVIsummer2022,MNDVIsummer2023,MNDVIwinter2018,MNDVIwinter2019,MNDVIwinter2020,MNDVIwinter2021,MNDVIwinter2022,MNDVIwinter2023,OSAVIsummer2018,OSAVIsummer2019,OSAVIsummer2020,OSAVIsummer2021,OSAVIsummer2022,OSAVIsummer2023,OSAVIwinter2018,OSAVIwinter2019,OSAVIwinter2020,OSAVIwinter2021,OSAVIwinter2022,OSAVIwinter2023,PVIsummer2018,PVIsummer2019,PVIsummer2020,PVIsummer2021,PVIsummer2022,PVIsummer2023,PVIwinter2018,PVIwinter2019,PVIwinter2020,PVIwinter2021,PVIwinter2022,PVIwinter2023,SARVIsummer2018,SARVIsummer2019,SARVIsummer2020,SARVIsummer2021,SARVIsummer2022,SARVIsummer2023,SARVIwinter2018,SARVIwinter2019,SARVIwinter2020,SARVIwinter2021,SARVIwinter2022,SARVIwinter2023,SLAVIsummer2018,SLAVIsummer2019,SLAVIsummer2020,SLAVIsummer2021,SLAVIsummer2022,SLAVIsummer2023,SLAVIwinter2018,SLAVIwinter2019,SLAVIwinter2020,SLAVIwinter2021,SLAVIwinter2022,SLAVIwinter2023,TSAVI2summer2018,TSAVI2summer2019,TSAVI2summer2020,TSAVI2summer2021,TSAVI2summer2022,TSAVI2summer2023,TSAVI2winter2018,TSAVI2winter2019,TSAVI2winter2020,TSAVI2winter2021,TSAVI2winter2022,TSAVI2winter2023,WDVIsummer2018,WDVIsummer2019,WDVIsummer2020,WDVIsummer2021,WDVIsummer2022,WDVIsummer2023,WDVIwinter

In [70]:
cov_df = pd.read_parquet(cov_path)
cov_df

,long,lat,species_names,sand_f,elevation,cfvo_f,phh2o_f,soc_f,slope,clay_f,nitrogen_f,cec_f,bdod_f,aspect,silt_f,ocd_f
0,75.66669580000007,32.25107380000003,Bambusoideae spp.,26.49250,352.0,13.5475,4.685,6.29375,13.422151,22.47375,8.6365,10.74375,1.008625,254.256500,26.03375,7.0275
1,75.66676390000003,32.25073910000003,Bambusoideae spp.,26.49250,350.0,13.5475,4.685,6.29375,10.147419,22.47375,8.6365,10.74375,1.008625,254.256454,26.03375,7.0275
2,75.66686810000004,32.25026770000005,Bambusoideae spp.,26.49250,349.0,13.5475,4.685,6.29375,8.110854,22.47375,8.6365,10.74375,1.008625,250.076767,26.03375,7.0275
3,75.66719240000003,32.249825100000066,Bambusoideae spp.,26.49250,351.0,13.5475,4.685,6.29375,9.826173,22.47375,8.6365,10.74375,1.008625,242.139969,26.03375,7.0275
4,75.66729090000007,32.24962370000003,Bambusoideae spp.,26.49250,348.0,13.5475,4.685,6.29375,6.125500,22.47375,8.6365,10.74375,1.008625,243.095016,26.03375,7.0275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148902,78.12497830000007,31.25865170000003,Rhododendron arboreum,36.73625,2309.0,19.9450,4.570,30.67875,54.879799,16.27250,11.7515,14.13875,0.968750,325.060272,21.98625,12.1675
148903,78.12522000000007,31.258880000000033,Rhododendron arboreum,36.73625,2292.0,19.9450,4.570,30.67875,58.185696,16.27250,11.7515,14.13875,0.968750,319.749359,21.98625,12.1675
148904,78.12614830000007,31.258660000000077,Rhododendron arboreum,36.73625,2361.0,19.9450,4.570,30.06625,30.714142,16.27250,11.7515,14.13875,0.966375,292.424896,21.98625,12.1675
148905,78.12618260000005,31.25882800000005,Rhododendron arboreum,36.73625,2350.0,19.9450,4.570,30.06625,43.765118,16.27250,11.7515,14.13875,0.966375,348.596802,21.98625,12.1675


In [71]:
merged_df = reduce(lambda left, right: pd.merge(left, right,  on=index_cols, how='outer', validate="1:1"), [s1, s2, cov_df])
merged_df



,long,lat,species_names,vv_Descendingsummer2018,vv_Descendingsummer2019,vv_Descendingsummer2020,vv_Descendingsummer2021,vv_Descendingsummer2022,vv_Descendingsummer2023,vv_Descendingwinter2018,vv_Descendingwinter2019,vv_Descendingwinter2020,vv_Descendingwinter2021,vv_Descendingwinter2022,vv_Descendingwinter2023,vh_Descendingsummer2018,vh_Descendingsummer2019,vh_Descendingsummer2020,vh_Descendingsummer2021,vh_Descendingsummer2022,vh_Descendingsummer2023,vh_Descendingwinter2018,vh_Descendingwinter2019,vh_Descendingwinter2020,vh_Descendingwinter2021,vh_Descendingwinter2022,vh_Descendingwinter2023,vv_Ascendingsummer2018,vv_Ascendingsummer2019,vv_Ascendingsummer2020,vv_Ascendingsummer2021,vv_Ascendingsummer2022,vv_Ascendingsummer2023,vv_Ascendingwinter2018,vv_Ascendingwinter2019,vv_Ascendingwinter2020,vv_Ascendingwinter2021,vv_Ascendingwinter2022,vv_Ascendingwinter2023,vh_Ascendingsummer2018,vh_Ascendingsummer2019,vh_Ascendingsummer2020,vh_Ascendingsummer2021,vh_Ascendingsummer2022,vh_Ascendingsummer2023,vh_Ascendingwinter2018,vh_Ascendingwinter2019,vh_Ascendingwinter2020,vh_Ascendingwinter2021,vh_Ascendingwinter2022,vh_Ascendingwinter2023,VV_VH_Asummer2018,VV_VH_Asummer2019,VV_VH_Asummer2020,VV_VH_Asummer2021,VV_VH_Asummer2022,VV_VH_Asummer2023,VV_VH_Awinter2018,VV_VH_Awinter2019,VV_VH_Awinter2020,VV_VH_Awinter2021,VV_VH_Awinter2022,VV_VH_Awinter2023,VV_VH_Dsummer2018,VV_VH_Dsummer2019,VV_VH_Dsummer2020,VV_VH_Dsummer2021,VV_VH_Dsummer2022,VV_VH_Dsummer2023,VV_VH_Dwinter2018,VV_VH_Dwinter2019,VV_VH_Dwinter2020,VV_VH_Dwinter2021,VV_VH_Dwinter2022,VV_VH_Dwinter2023,VH_VV_Asummer2018,VH_VV_Asummer2019,VH_VV_Asummer2020,VH_VV_Asummer2021,VH_VV_Asummer2022,VH_VV_Asummer2023,VH_VV_Awinter2018,VH_VV_Awinter2019,VH_VV_Awinter2020,VH_VV_Awinter2021,VH_VV_Awinter2022,VH_VV_Awinter2023,VH_VV_Dsummer2018,VH_VV_Dsummer2019,VH_VV_Dsummer2020,VH_VV_Dsummer2021,VH_VV_Dsummer2022,VH_VV_Dsummer2023,VH_VV_Dwinter2018,VH_VV_Dwinter2019,VH_VV_Dwinter2020,VH_VV_Dwinter2021,VH_VV_Dwinter2022,VH_VV_Dwinter2023,SAR_NDVI_Asummer2018,SAR_NDVI_Asummer2019,SAR_NDVI_Asummer2020,SAR_NDVI_Asummer2021,SAR_NDVI_Asummer2022,SAR_NDVI_Asummer2023,SAR_NDVI_Awinter2018,SAR_NDVI_Awinter2019,SAR_NDVI_Awinter2020,SAR_NDVI_Awinter2021,SAR_NDVI_Awinter2022,SAR_NDVI_Awinter2023,SAR_NDVI_Dsummer2018,SAR_NDVI_Dsummer2019,SAR_NDVI_Dsummer2020,SAR_NDVI_Dsummer2021,SAR_NDVI_Dsummer2022,SAR_NDVI_Dsummer2023,SAR_NDVI_Dwinter2018,SAR_NDVI_Dwinter2019,SAR_NDVI_Dwinter2020,SAR_NDVI_Dwinter2021,SAR_NDVI_Dwinter2022,SAR_NDVI_Dwinter2023,DVI_Asummer2018,DVI_Asummer2019,DVI_Asummer2020,DVI_Asummer2021,DVI_Asummer2022,DVI_Asummer2023,DVI_Awinter2018,DVI_Awinter2019,DVI_Awinter2020,DVI_Awinter2021,DVI_Awinter2022,DVI_Awinter2023,DVI_Dsummer2018,DVI_Dsummer2019,DVI_Dsummer2020,DVI_Dsummer2021,DVI_Dsummer2022,DVI_Dsummer2023,DVI_Dwinter2018,DVI_Dwinter2019,DVI_Dwinter2020,DVI_Dwinter2021,DVI_Dwinter2022,DVI_Dwinter2023,SVI_Asummer2018,SVI_Asummer2019,SVI_Asummer2020,SVI_Asummer2021,SVI_Asummer2022,SVI_Asummer2023,SVI_Awinter2018,SVI_Awinter2019,SVI_Awinter2020,SVI_Awinter2021,SVI_Awinter2022,SVI_Awinter2023,SVI_Dsummer2018,SVI_Dsummer2019,SVI_Dsummer2020,SVI_Dsummer2021,SVI_Dsummer2022,SVI_Dsummer2023,SVI_Dwinter2018,SVI_Dwinter2019,SVI_Dwinter2020,SVI_Dwinter2021,SVI_Dwinter2022,SVI_Dwinter2023,RDVI_Asummer2018,RDVI_Asummer2019,RDVI_Asummer2020,RDVI_Asummer2021,RDVI_Asummer2022,RDVI_Asummer2023,RDVI_Awinter2018,RDVI_Awinter2019,RDVI_Awinter2020,RDVI_Awinter2021,RDVI_Awinter2022,RDVI_Awinter2023,RDVI_Dsummer2018,RDVI_Dsummer2019,RDVI_Dsummer2020,RDVI_Dsummer2021,RDVI_Dsummer2022,RDVI_Dsummer2023,RDVI_Dwinter2018,RDVI_Dwinter2019,RDVI_Dwinter2020,RDVI_Dwinter2021,RDVI_Dwinter2022,RDVI_Dwinter2023,NRDVI_Asummer2018,NRDVI_Asummer2019,NRDVI_Asummer2020,NRDVI_Asummer2021,NRDVI_Asummer2022,NRDVI_Asummer2023,NRDVI_Awinter2018,NRDVI_Awinter2019,NRDVI_Awinter2020,NRDVI_Awinter2021,NRDVI_Awinter2022,NRDVI_Awinter2023,NRDVI_Dsummer2018,NRDVI_Dsummer2019,NRDVI_Dsummer2020,NRDVI_Dsummer2021,NRDVI_Dsummer202

### Species Codes and JSON File

In [72]:
if not species_dict_path.exists():

    #exporting species dictionary as json
    cats = pd.Categorical(merged_df['species_names'])
    species_dict = dict(zip(range(len(cats.categories)), cats.categories))

    with open(species_dict_path, 'w') as json_file:
        json.dump(species_dict, json_file, indent=4)

In [73]:
merged_df["species"] = pd.Categorical(merged_df['species_names']).codes
merged_df.drop('species_names', axis=1, inplace=True)
merged_df

,long,lat,vv_Descendingsummer2018,vv_Descendingsummer2019,vv_Descendingsummer2020,vv_Descendingsummer2021,vv_Descendingsummer2022,vv_Descendingsummer2023,vv_Descendingwinter2018,vv_Descendingwinter2019,vv_Descendingwinter2020,vv_Descendingwinter2021,vv_Descendingwinter2022,vv_Descendingwinter2023,vh_Descendingsummer2018,vh_Descendingsummer2019,vh_Descendingsummer2020,vh_Descendingsummer2021,vh_Descendingsummer2022,vh_Descendingsummer2023,vh_Descendingwinter2018,vh_Descendingwinter2019,vh_Descendingwinter2020,vh_Descendingwinter2021,vh_Descendingwinter2022,vh_Descendingwinter2023,vv_Ascendingsummer2018,vv_Ascendingsummer2019,vv_Ascendingsummer2020,vv_Ascendingsummer2021,vv_Ascendingsummer2022,vv_Ascendingsummer2023,vv_Ascendingwinter2018,vv_Ascendingwinter2019,vv_Ascendingwinter2020,vv_Ascendingwinter2021,vv_Ascendingwinter2022,vv_Ascendingwinter2023,vh_Ascendingsummer2018,vh_Ascendingsummer2019,vh_Ascendingsummer2020,vh_Ascendingsummer2021,vh_Ascendingsummer2022,vh_Ascendingsummer2023,vh_Ascendingwinter2018,vh_Ascendingwinter2019,vh_Ascendingwinter2020,vh_Ascendingwinter2021,vh_Ascendingwinter2022,vh_Ascendingwinter2023,VV_VH_Asummer2018,VV_VH_Asummer2019,VV_VH_Asummer2020,VV_VH_Asummer2021,VV_VH_Asummer2022,VV_VH_Asummer2023,VV_VH_Awinter2018,VV_VH_Awinter2019,VV_VH_Awinter2020,VV_VH_Awinter2021,VV_VH_Awinter2022,VV_VH_Awinter2023,VV_VH_Dsummer2018,VV_VH_Dsummer2019,VV_VH_Dsummer2020,VV_VH_Dsummer2021,VV_VH_Dsummer2022,VV_VH_Dsummer2023,VV_VH_Dwinter2018,VV_VH_Dwinter2019,VV_VH_Dwinter2020,VV_VH_Dwinter2021,VV_VH_Dwinter2022,VV_VH_Dwinter2023,VH_VV_Asummer2018,VH_VV_Asummer2019,VH_VV_Asummer2020,VH_VV_Asummer2021,VH_VV_Asummer2022,VH_VV_Asummer2023,VH_VV_Awinter2018,VH_VV_Awinter2019,VH_VV_Awinter2020,VH_VV_Awinter2021,VH_VV_Awinter2022,VH_VV_Awinter2023,VH_VV_Dsummer2018,VH_VV_Dsummer2019,VH_VV_Dsummer2020,VH_VV_Dsummer2021,VH_VV_Dsummer2022,VH_VV_Dsummer2023,VH_VV_Dwinter2018,VH_VV_Dwinter2019,VH_VV_Dwinter2020,VH_VV_Dwinter2021,VH_VV_Dwinter2022,VH_VV_Dwinter2023,SAR_NDVI_Asummer2018,SAR_NDVI_Asummer2019,SAR_NDVI_Asummer2020,SAR_NDVI_Asummer2021,SAR_NDVI_Asummer2022,SAR_NDVI_Asummer2023,SAR_NDVI_Awinter2018,SAR_NDVI_Awinter2019,SAR_NDVI_Awinter2020,SAR_NDVI_Awinter2021,SAR_NDVI_Awinter2022,SAR_NDVI_Awinter2023,SAR_NDVI_Dsummer2018,SAR_NDVI_Dsummer2019,SAR_NDVI_Dsummer2020,SAR_NDVI_Dsummer2021,SAR_NDVI_Dsummer2022,SAR_NDVI_Dsummer2023,SAR_NDVI_Dwinter2018,SAR_NDVI_Dwinter2019,SAR_NDVI_Dwinter2020,SAR_NDVI_Dwinter2021,SAR_NDVI_Dwinter2022,SAR_NDVI_Dwinter2023,DVI_Asummer2018,DVI_Asummer2019,DVI_Asummer2020,DVI_Asummer2021,DVI_Asummer2022,DVI_Asummer2023,DVI_Awinter2018,DVI_Awinter2019,DVI_Awinter2020,DVI_Awinter2021,DVI_Awinter2022,DVI_Awinter2023,DVI_Dsummer2018,DVI_Dsummer2019,DVI_Dsummer2020,DVI_Dsummer2021,DVI_Dsummer2022,DVI_Dsummer2023,DVI_Dwinter2018,DVI_Dwinter2019,DVI_Dwinter2020,DVI_Dwinter2021,DVI_Dwinter2022,DVI_Dwinter2023,SVI_Asummer2018,SVI_Asummer2019,SVI_Asummer2020,SVI_Asummer2021,SVI_Asummer2022,SVI_Asummer2023,SVI_Awinter2018,SVI_Awinter2019,SVI_Awinter2020,SVI_Awinter2021,SVI_Awinter2022,SVI_Awinter2023,SVI_Dsummer2018,SVI_Dsummer2019,SVI_Dsummer2020,SVI_Dsummer2021,SVI_Dsummer2022,SVI_Dsummer2023,SVI_Dwinter2018,SVI_Dwinter2019,SVI_Dwinter2020,SVI_Dwinter2021,SVI_Dwinter2022,SVI_Dwinter2023,RDVI_Asummer2018,RDVI_Asummer2019,RDVI_Asummer2020,RDVI_Asummer2021,RDVI_Asummer2022,RDVI_Asummer2023,RDVI_Awinter2018,RDVI_Awinter2019,RDVI_Awinter2020,RDVI_Awinter2021,RDVI_Awinter2022,RDVI_Awinter2023,RDVI_Dsummer2018,RDVI_Dsummer2019,RDVI_Dsummer2020,RDVI_Dsummer2021,RDVI_Dsummer2022,RDVI_Dsummer2023,RDVI_Dwinter2018,RDVI_Dwinter2019,RDVI_Dwinter2020,RDVI_Dwinter2021,RDVI_Dwinter2022,RDVI_Dwinter2023,NRDVI_Asummer2018,NRDVI_Asummer2019,NRDVI_Asummer2020,NRDVI_Asummer2021,NRDVI_Asummer2022,NRDVI_Asummer2023,NRDVI_Awinter2018,NRDVI_Awinter2019,NRDVI_Awinter2020,NRDVI_Awinter2021,NRDVI_Awinter2022,NRDVI_Awinter2023,NRDVI_Dsummer2018,NRDVI_Dsummer2019,NRDVI_Dsummer2020,NRDVI_Dsummer2021,NRDVI_Dsummer2022,NRDVI_Dsumme

### Normalizing

In [74]:
df_index_cols = merged_df[['long', 'lat', 'species']]
df_x = merged_df.drop(['long', 'lat', 'species'], axis=1)

df_x

,vv_Descendingsummer2018,vv_Descendingsummer2019,vv_Descendingsummer2020,vv_Descendingsummer2021,vv_Descendingsummer2022,vv_Descendingsummer2023,vv_Descendingwinter2018,vv_Descendingwinter2019,vv_Descendingwinter2020,vv_Descendingwinter2021,vv_Descendingwinter2022,vv_Descendingwinter2023,vh_Descendingsummer2018,vh_Descendingsummer2019,vh_Descendingsummer2020,vh_Descendingsummer2021,vh_Descendingsummer2022,vh_Descendingsummer2023,vh_Descendingwinter2018,vh_Descendingwinter2019,vh_Descendingwinter2020,vh_Descendingwinter2021,vh_Descendingwinter2022,vh_Descendingwinter2023,vv_Ascendingsummer2018,vv_Ascendingsummer2019,vv_Ascendingsummer2020,vv_Ascendingsummer2021,vv_Ascendingsummer2022,vv_Ascendingsummer2023,vv_Ascendingwinter2018,vv_Ascendingwinter2019,vv_Ascendingwinter2020,vv_Ascendingwinter2021,vv_Ascendingwinter2022,vv_Ascendingwinter2023,vh_Ascendingsummer2018,vh_Ascendingsummer2019,vh_Ascendingsummer2020,vh_Ascendingsummer2021,vh_Ascendingsummer2022,vh_Ascendingsummer2023,vh_Ascendingwinter2018,vh_Ascendingwinter2019,vh_Ascendingwinter2020,vh_Ascendingwinter2021,vh_Ascendingwinter2022,vh_Ascendingwinter2023,VV_VH_Asummer2018,VV_VH_Asummer2019,VV_VH_Asummer2020,VV_VH_Asummer2021,VV_VH_Asummer2022,VV_VH_Asummer2023,VV_VH_Awinter2018,VV_VH_Awinter2019,VV_VH_Awinter2020,VV_VH_Awinter2021,VV_VH_Awinter2022,VV_VH_Awinter2023,VV_VH_Dsummer2018,VV_VH_Dsummer2019,VV_VH_Dsummer2020,VV_VH_Dsummer2021,VV_VH_Dsummer2022,VV_VH_Dsummer2023,VV_VH_Dwinter2018,VV_VH_Dwinter2019,VV_VH_Dwinter2020,VV_VH_Dwinter2021,VV_VH_Dwinter2022,VV_VH_Dwinter2023,VH_VV_Asummer2018,VH_VV_Asummer2019,VH_VV_Asummer2020,VH_VV_Asummer2021,VH_VV_Asummer2022,VH_VV_Asummer2023,VH_VV_Awinter2018,VH_VV_Awinter2019,VH_VV_Awinter2020,VH_VV_Awinter2021,VH_VV_Awinter2022,VH_VV_Awinter2023,VH_VV_Dsummer2018,VH_VV_Dsummer2019,VH_VV_Dsummer2020,VH_VV_Dsummer2021,VH_VV_Dsummer2022,VH_VV_Dsummer2023,VH_VV_Dwinter2018,VH_VV_Dwinter2019,VH_VV_Dwinter2020,VH_VV_Dwinter2021,VH_VV_Dwinter2022,VH_VV_Dwinter2023,SAR_NDVI_Asummer2018,SAR_NDVI_Asummer2019,SAR_NDVI_Asummer2020,SAR_NDVI_Asummer2021,SAR_NDVI_Asummer2022,SAR_NDVI_Asummer2023,SAR_NDVI_Awinter2018,SAR_NDVI_Awinter2019,SAR_NDVI_Awinter2020,SAR_NDVI_Awinter2021,SAR_NDVI_Awinter2022,SAR_NDVI_Awinter2023,SAR_NDVI_Dsummer2018,SAR_NDVI_Dsummer2019,SAR_NDVI_Dsummer2020,SAR_NDVI_Dsummer2021,SAR_NDVI_Dsummer2022,SAR_NDVI_Dsummer2023,SAR_NDVI_Dwinter2018,SAR_NDVI_Dwinter2019,SAR_NDVI_Dwinter2020,SAR_NDVI_Dwinter2021,SAR_NDVI_Dwinter2022,SAR_NDVI_Dwinter2023,DVI_Asummer2018,DVI_Asummer2019,DVI_Asummer2020,DVI_Asummer2021,DVI_Asummer2022,DVI_Asummer2023,DVI_Awinter2018,DVI_Awinter2019,DVI_Awinter2020,DVI_Awinter2021,DVI_Awinter2022,DVI_Awinter2023,DVI_Dsummer2018,DVI_Dsummer2019,DVI_Dsummer2020,DVI_Dsummer2021,DVI_Dsummer2022,DVI_Dsummer2023,DVI_Dwinter2018,DVI_Dwinter2019,DVI_Dwinter2020,DVI_Dwinter2021,DVI_Dwinter2022,DVI_Dwinter2023,SVI_Asummer2018,SVI_Asummer2019,SVI_Asummer2020,SVI_Asummer2021,SVI_Asummer2022,SVI_Asummer2023,SVI_Awinter2018,SVI_Awinter2019,SVI_Awinter2020,SVI_Awinter2021,SVI_Awinter2022,SVI_Awinter2023,SVI_Dsummer2018,SVI_Dsummer2019,SVI_Dsummer2020,SVI_Dsummer2021,SVI_Dsummer2022,SVI_Dsummer2023,SVI_Dwinter2018,SVI_Dwinter2019,SVI_Dwinter2020,SVI_Dwinter2021,SVI_Dwinter2022,SVI_Dwinter2023,RDVI_Asummer2018,RDVI_Asummer2019,RDVI_Asummer2020,RDVI_Asummer2021,RDVI_Asummer2022,RDVI_Asummer2023,RDVI_Awinter2018,RDVI_Awinter2019,RDVI_Awinter2020,RDVI_Awinter2021,RDVI_Awinter2022,RDVI_Awinter2023,RDVI_Dsummer2018,RDVI_Dsummer2019,RDVI_Dsummer2020,RDVI_Dsummer2021,RDVI_Dsummer2022,RDVI_Dsummer2023,RDVI_Dwinter2018,RDVI_Dwinter2019,RDVI_Dwinter2020,RDVI_Dwinter2021,RDVI_Dwinter2022,RDVI_Dwinter2023,NRDVI_Asummer2018,NRDVI_Asummer2019,NRDVI_Asummer2020,NRDVI_Asummer2021,NRDVI_Asummer2022,NRDVI_Asummer2023,NRDVI_Awinter2018,NRDVI_Awinter2019,NRDVI_Awinter2020,NRDVI_Awinter2021,NRDVI_Awinter2022,NRDVI_Awinter2023,NRDVI_Dsummer2018,NRDVI_Dsummer2019,NRDVI_Dsummer2020,NRDVI_Dsummer2021,NRDVI_Dsummer2022,NRDVI_Dsummer2023,NRD

In [75]:
min_max_scaler = MinMaxScaler()

df_scaled = pd.DataFrame(min_max_scaler.fit_transform(df_x.values), columns=df_x.columns)
df_scaled

,vv_Descendingsummer2018,vv_Descendingsummer2019,vv_Descendingsummer2020,vv_Descendingsummer2021,vv_Descendingsummer2022,vv_Descendingsummer2023,vv_Descendingwinter2018,vv_Descendingwinter2019,vv_Descendingwinter2020,vv_Descendingwinter2021,vv_Descendingwinter2022,vv_Descendingwinter2023,vh_Descendingsummer2018,vh_Descendingsummer2019,vh_Descendingsummer2020,vh_Descendingsummer2021,vh_Descendingsummer2022,vh_Descendingsummer2023,vh_Descendingwinter2018,vh_Descendingwinter2019,vh_Descendingwinter2020,vh_Descendingwinter2021,vh_Descendingwinter2022,vh_Descendingwinter2023,vv_Ascendingsummer2018,vv_Ascendingsummer2019,vv_Ascendingsummer2020,vv_Ascendingsummer2021,vv_Ascendingsummer2022,vv_Ascendingsummer2023,vv_Ascendingwinter2018,vv_Ascendingwinter2019,vv_Ascendingwinter2020,vv_Ascendingwinter2021,vv_Ascendingwinter2022,vv_Ascendingwinter2023,vh_Ascendingsummer2018,vh_Ascendingsummer2019,vh_Ascendingsummer2020,vh_Ascendingsummer2021,vh_Ascendingsummer2022,vh_Ascendingsummer2023,vh_Ascendingwinter2018,vh_Ascendingwinter2019,vh_Ascendingwinter2020,vh_Ascendingwinter2021,vh_Ascendingwinter2022,vh_Ascendingwinter2023,VV_VH_Asummer2018,VV_VH_Asummer2019,VV_VH_Asummer2020,VV_VH_Asummer2021,VV_VH_Asummer2022,VV_VH_Asummer2023,VV_VH_Awinter2018,VV_VH_Awinter2019,VV_VH_Awinter2020,VV_VH_Awinter2021,VV_VH_Awinter2022,VV_VH_Awinter2023,VV_VH_Dsummer2018,VV_VH_Dsummer2019,VV_VH_Dsummer2020,VV_VH_Dsummer2021,VV_VH_Dsummer2022,VV_VH_Dsummer2023,VV_VH_Dwinter2018,VV_VH_Dwinter2019,VV_VH_Dwinter2020,VV_VH_Dwinter2021,VV_VH_Dwinter2022,VV_VH_Dwinter2023,VH_VV_Asummer2018,VH_VV_Asummer2019,VH_VV_Asummer2020,VH_VV_Asummer2021,VH_VV_Asummer2022,VH_VV_Asummer2023,VH_VV_Awinter2018,VH_VV_Awinter2019,VH_VV_Awinter2020,VH_VV_Awinter2021,VH_VV_Awinter2022,VH_VV_Awinter2023,VH_VV_Dsummer2018,VH_VV_Dsummer2019,VH_VV_Dsummer2020,VH_VV_Dsummer2021,VH_VV_Dsummer2022,VH_VV_Dsummer2023,VH_VV_Dwinter2018,VH_VV_Dwinter2019,VH_VV_Dwinter2020,VH_VV_Dwinter2021,VH_VV_Dwinter2022,VH_VV_Dwinter2023,SAR_NDVI_Asummer2018,SAR_NDVI_Asummer2019,SAR_NDVI_Asummer2020,SAR_NDVI_Asummer2021,SAR_NDVI_Asummer2022,SAR_NDVI_Asummer2023,SAR_NDVI_Awinter2018,SAR_NDVI_Awinter2019,SAR_NDVI_Awinter2020,SAR_NDVI_Awinter2021,SAR_NDVI_Awinter2022,SAR_NDVI_Awinter2023,SAR_NDVI_Dsummer2018,SAR_NDVI_Dsummer2019,SAR_NDVI_Dsummer2020,SAR_NDVI_Dsummer2021,SAR_NDVI_Dsummer2022,SAR_NDVI_Dsummer2023,SAR_NDVI_Dwinter2018,SAR_NDVI_Dwinter2019,SAR_NDVI_Dwinter2020,SAR_NDVI_Dwinter2021,SAR_NDVI_Dwinter2022,SAR_NDVI_Dwinter2023,DVI_Asummer2018,DVI_Asummer2019,DVI_Asummer2020,DVI_Asummer2021,DVI_Asummer2022,DVI_Asummer2023,DVI_Awinter2018,DVI_Awinter2019,DVI_Awinter2020,DVI_Awinter2021,DVI_Awinter2022,DVI_Awinter2023,DVI_Dsummer2018,DVI_Dsummer2019,DVI_Dsummer2020,DVI_Dsummer2021,DVI_Dsummer2022,DVI_Dsummer2023,DVI_Dwinter2018,DVI_Dwinter2019,DVI_Dwinter2020,DVI_Dwinter2021,DVI_Dwinter2022,DVI_Dwinter2023,SVI_Asummer2018,SVI_Asummer2019,SVI_Asummer2020,SVI_Asummer2021,SVI_Asummer2022,SVI_Asummer2023,SVI_Awinter2018,SVI_Awinter2019,SVI_Awinter2020,SVI_Awinter2021,SVI_Awinter2022,SVI_Awinter2023,SVI_Dsummer2018,SVI_Dsummer2019,SVI_Dsummer2020,SVI_Dsummer2021,SVI_Dsummer2022,SVI_Dsummer2023,SVI_Dwinter2018,SVI_Dwinter2019,SVI_Dwinter2020,SVI_Dwinter2021,SVI_Dwinter2022,SVI_Dwinter2023,RDVI_Asummer2018,RDVI_Asummer2019,RDVI_Asummer2020,RDVI_Asummer2021,RDVI_Asummer2022,RDVI_Asummer2023,RDVI_Awinter2018,RDVI_Awinter2019,RDVI_Awinter2020,RDVI_Awinter2021,RDVI_Awinter2022,RDVI_Awinter2023,RDVI_Dsummer2018,RDVI_Dsummer2019,RDVI_Dsummer2020,RDVI_Dsummer2021,RDVI_Dsummer2022,RDVI_Dsummer2023,RDVI_Dwinter2018,RDVI_Dwinter2019,RDVI_Dwinter2020,RDVI_Dwinter2021,RDVI_Dwinter2022,RDVI_Dwinter2023,NRDVI_Asummer2018,NRDVI_Asummer2019,NRDVI_Asummer2020,NRDVI_Asummer2021,NRDVI_Asummer2022,NRDVI_Asummer2023,NRDVI_Awinter2018,NRDVI_Awinter2019,NRDVI_Awinter2020,NRDVI_Awinter2021,NRDVI_Awinter2022,NRDVI_Awinter2023,NRDVI_Dsummer2018,NRDVI_Dsummer2019,NRDVI_Dsummer2020,NRDVI_Dsummer2021,NRDVI_Dsummer2022,NRDVI_Dsummer2023,NRD

### Model Data Frame

In [76]:
model_df = pd.concat([df_index_cols, df_scaled], axis=1)
model_df

,long,lat,species,vv_Descendingsummer2018,vv_Descendingsummer2019,vv_Descendingsummer2020,vv_Descendingsummer2021,vv_Descendingsummer2022,vv_Descendingsummer2023,vv_Descendingwinter2018,vv_Descendingwinter2019,vv_Descendingwinter2020,vv_Descendingwinter2021,vv_Descendingwinter2022,vv_Descendingwinter2023,vh_Descendingsummer2018,vh_Descendingsummer2019,vh_Descendingsummer2020,vh_Descendingsummer2021,vh_Descendingsummer2022,vh_Descendingsummer2023,vh_Descendingwinter2018,vh_Descendingwinter2019,vh_Descendingwinter2020,vh_Descendingwinter2021,vh_Descendingwinter2022,vh_Descendingwinter2023,vv_Ascendingsummer2018,vv_Ascendingsummer2019,vv_Ascendingsummer2020,vv_Ascendingsummer2021,vv_Ascendingsummer2022,vv_Ascendingsummer2023,vv_Ascendingwinter2018,vv_Ascendingwinter2019,vv_Ascendingwinter2020,vv_Ascendingwinter2021,vv_Ascendingwinter2022,vv_Ascendingwinter2023,vh_Ascendingsummer2018,vh_Ascendingsummer2019,vh_Ascendingsummer2020,vh_Ascendingsummer2021,vh_Ascendingsummer2022,vh_Ascendingsummer2023,vh_Ascendingwinter2018,vh_Ascendingwinter2019,vh_Ascendingwinter2020,vh_Ascendingwinter2021,vh_Ascendingwinter2022,vh_Ascendingwinter2023,VV_VH_Asummer2018,VV_VH_Asummer2019,VV_VH_Asummer2020,VV_VH_Asummer2021,VV_VH_Asummer2022,VV_VH_Asummer2023,VV_VH_Awinter2018,VV_VH_Awinter2019,VV_VH_Awinter2020,VV_VH_Awinter2021,VV_VH_Awinter2022,VV_VH_Awinter2023,VV_VH_Dsummer2018,VV_VH_Dsummer2019,VV_VH_Dsummer2020,VV_VH_Dsummer2021,VV_VH_Dsummer2022,VV_VH_Dsummer2023,VV_VH_Dwinter2018,VV_VH_Dwinter2019,VV_VH_Dwinter2020,VV_VH_Dwinter2021,VV_VH_Dwinter2022,VV_VH_Dwinter2023,VH_VV_Asummer2018,VH_VV_Asummer2019,VH_VV_Asummer2020,VH_VV_Asummer2021,VH_VV_Asummer2022,VH_VV_Asummer2023,VH_VV_Awinter2018,VH_VV_Awinter2019,VH_VV_Awinter2020,VH_VV_Awinter2021,VH_VV_Awinter2022,VH_VV_Awinter2023,VH_VV_Dsummer2018,VH_VV_Dsummer2019,VH_VV_Dsummer2020,VH_VV_Dsummer2021,VH_VV_Dsummer2022,VH_VV_Dsummer2023,VH_VV_Dwinter2018,VH_VV_Dwinter2019,VH_VV_Dwinter2020,VH_VV_Dwinter2021,VH_VV_Dwinter2022,VH_VV_Dwinter2023,SAR_NDVI_Asummer2018,SAR_NDVI_Asummer2019,SAR_NDVI_Asummer2020,SAR_NDVI_Asummer2021,SAR_NDVI_Asummer2022,SAR_NDVI_Asummer2023,SAR_NDVI_Awinter2018,SAR_NDVI_Awinter2019,SAR_NDVI_Awinter2020,SAR_NDVI_Awinter2021,SAR_NDVI_Awinter2022,SAR_NDVI_Awinter2023,SAR_NDVI_Dsummer2018,SAR_NDVI_Dsummer2019,SAR_NDVI_Dsummer2020,SAR_NDVI_Dsummer2021,SAR_NDVI_Dsummer2022,SAR_NDVI_Dsummer2023,SAR_NDVI_Dwinter2018,SAR_NDVI_Dwinter2019,SAR_NDVI_Dwinter2020,SAR_NDVI_Dwinter2021,SAR_NDVI_Dwinter2022,SAR_NDVI_Dwinter2023,DVI_Asummer2018,DVI_Asummer2019,DVI_Asummer2020,DVI_Asummer2021,DVI_Asummer2022,DVI_Asummer2023,DVI_Awinter2018,DVI_Awinter2019,DVI_Awinter2020,DVI_Awinter2021,DVI_Awinter2022,DVI_Awinter2023,DVI_Dsummer2018,DVI_Dsummer2019,DVI_Dsummer2020,DVI_Dsummer2021,DVI_Dsummer2022,DVI_Dsummer2023,DVI_Dwinter2018,DVI_Dwinter2019,DVI_Dwinter2020,DVI_Dwinter2021,DVI_Dwinter2022,DVI_Dwinter2023,SVI_Asummer2018,SVI_Asummer2019,SVI_Asummer2020,SVI_Asummer2021,SVI_Asummer2022,SVI_Asummer2023,SVI_Awinter2018,SVI_Awinter2019,SVI_Awinter2020,SVI_Awinter2021,SVI_Awinter2022,SVI_Awinter2023,SVI_Dsummer2018,SVI_Dsummer2019,SVI_Dsummer2020,SVI_Dsummer2021,SVI_Dsummer2022,SVI_Dsummer2023,SVI_Dwinter2018,SVI_Dwinter2019,SVI_Dwinter2020,SVI_Dwinter2021,SVI_Dwinter2022,SVI_Dwinter2023,RDVI_Asummer2018,RDVI_Asummer2019,RDVI_Asummer2020,RDVI_Asummer2021,RDVI_Asummer2022,RDVI_Asummer2023,RDVI_Awinter2018,RDVI_Awinter2019,RDVI_Awinter2020,RDVI_Awinter2021,RDVI_Awinter2022,RDVI_Awinter2023,RDVI_Dsummer2018,RDVI_Dsummer2019,RDVI_Dsummer2020,RDVI_Dsummer2021,RDVI_Dsummer2022,RDVI_Dsummer2023,RDVI_Dwinter2018,RDVI_Dwinter2019,RDVI_Dwinter2020,RDVI_Dwinter2021,RDVI_Dwinter2022,RDVI_Dwinter2023,NRDVI_Asummer2018,NRDVI_Asummer2019,NRDVI_Asummer2020,NRDVI_Asummer2021,NRDVI_Asummer2022,NRDVI_Asummer2023,NRDVI_Awinter2018,NRDVI_Awinter2019,NRDVI_Awinter2020,NRDVI_Awinter2021,NRDVI_Awinter2022,NRDVI_Awinter2023,NRDVI_Dsummer2018,NRDVI_Dsummer2019,NRDVI_Dsummer2020,NRDVI_Dsummer2021,NRDVI_Dsummer2022,NRDV

In [77]:
for col in model_df.columns:
    print(pd.Series(model_df[col].isna().value_counts()))

long
False    148907
Name: count, dtype: int64
lat
False    148907
Name: count, dtype: int64
species
False    148907
Name: count, dtype: int64
vv_Descendingsummer2018
False    148907
Name: count, dtype: int64
vv_Descendingsummer2019
False    148907
Name: count, dtype: int64
vv_Descendingsummer2020
False    148907
Name: count, dtype: int64
vv_Descendingsummer2021
False    148907
Name: count, dtype: int64
vv_Descendingsummer2022
False    148907
Name: count, dtype: int64
vv_Descendingsummer2023
False    148907
Name: count, dtype: int64
vv_Descendingwinter2018
False    148907
Name: count, dtype: int64
vv_Descendingwinter2019
False    148907
Name: count, dtype: int64
vv_Descendingwinter2020
False    148907
Name: count, dtype: int64
vv_Descendingwinter2021
False    148907
Name: count, dtype: int64
vv_Descendingwinter2022
False    148907
Name: count, dtype: int64
vv_Descendingwinter2023
False    148907
Name: count, dtype: int64
vh_Descendingsummer2018
False    148907
Name: count, dtype: int64

### Balanced Panel of Species

For seven species, (13,3,1,0,5,10,6) which have locations less than 5000, we have taken all the locations, and for the remaining 7, we have randomly sampled 5000 locations brininging the total observations to 64,147

In [78]:
model_df['species'].value_counts().sort_values()

species
13     3142
3      3524
1      3939
0      4206
5      4737
10     4765
6      4834
7      6191
9      6335
4      8513
12    14398
2     20625
11    22990
8     40708
Name: count, dtype: int64

In [79]:
set_seed = 4526
species_df_holder = []
for species_id, species_df in model_df.groupby('species'):
    sampled_species_df = []

    if species_id in [13,3,1,0,5,10,6]:
        sampled_species_df = species_df.reset_index(drop=True)
    
    elif species_id in [7,9,4,12,2,11,8]:
        sampled_species_df = species_df.sample(n=5000, random_state=set_seed).reset_index(drop=True)

    species_df_holder.append(sampled_species_df)

species_model_df = pd.concat(species_df_holder, axis=0)
species_model_df

,long,lat,species,vv_Descendingsummer2018,vv_Descendingsummer2019,vv_Descendingsummer2020,vv_Descendingsummer2021,vv_Descendingsummer2022,vv_Descendingsummer2023,vv_Descendingwinter2018,vv_Descendingwinter2019,vv_Descendingwinter2020,vv_Descendingwinter2021,vv_Descendingwinter2022,vv_Descendingwinter2023,vh_Descendingsummer2018,vh_Descendingsummer2019,vh_Descendingsummer2020,vh_Descendingsummer2021,vh_Descendingsummer2022,vh_Descendingsummer2023,vh_Descendingwinter2018,vh_Descendingwinter2019,vh_Descendingwinter2020,vh_Descendingwinter2021,vh_Descendingwinter2022,vh_Descendingwinter2023,vv_Ascendingsummer2018,vv_Ascendingsummer2019,vv_Ascendingsummer2020,vv_Ascendingsummer2021,vv_Ascendingsummer2022,vv_Ascendingsummer2023,vv_Ascendingwinter2018,vv_Ascendingwinter2019,vv_Ascendingwinter2020,vv_Ascendingwinter2021,vv_Ascendingwinter2022,vv_Ascendingwinter2023,vh_Ascendingsummer2018,vh_Ascendingsummer2019,vh_Ascendingsummer2020,vh_Ascendingsummer2021,vh_Ascendingsummer2022,vh_Ascendingsummer2023,vh_Ascendingwinter2018,vh_Ascendingwinter2019,vh_Ascendingwinter2020,vh_Ascendingwinter2021,vh_Ascendingwinter2022,vh_Ascendingwinter2023,VV_VH_Asummer2018,VV_VH_Asummer2019,VV_VH_Asummer2020,VV_VH_Asummer2021,VV_VH_Asummer2022,VV_VH_Asummer2023,VV_VH_Awinter2018,VV_VH_Awinter2019,VV_VH_Awinter2020,VV_VH_Awinter2021,VV_VH_Awinter2022,VV_VH_Awinter2023,VV_VH_Dsummer2018,VV_VH_Dsummer2019,VV_VH_Dsummer2020,VV_VH_Dsummer2021,VV_VH_Dsummer2022,VV_VH_Dsummer2023,VV_VH_Dwinter2018,VV_VH_Dwinter2019,VV_VH_Dwinter2020,VV_VH_Dwinter2021,VV_VH_Dwinter2022,VV_VH_Dwinter2023,VH_VV_Asummer2018,VH_VV_Asummer2019,VH_VV_Asummer2020,VH_VV_Asummer2021,VH_VV_Asummer2022,VH_VV_Asummer2023,VH_VV_Awinter2018,VH_VV_Awinter2019,VH_VV_Awinter2020,VH_VV_Awinter2021,VH_VV_Awinter2022,VH_VV_Awinter2023,VH_VV_Dsummer2018,VH_VV_Dsummer2019,VH_VV_Dsummer2020,VH_VV_Dsummer2021,VH_VV_Dsummer2022,VH_VV_Dsummer2023,VH_VV_Dwinter2018,VH_VV_Dwinter2019,VH_VV_Dwinter2020,VH_VV_Dwinter2021,VH_VV_Dwinter2022,VH_VV_Dwinter2023,SAR_NDVI_Asummer2018,SAR_NDVI_Asummer2019,SAR_NDVI_Asummer2020,SAR_NDVI_Asummer2021,SAR_NDVI_Asummer2022,SAR_NDVI_Asummer2023,SAR_NDVI_Awinter2018,SAR_NDVI_Awinter2019,SAR_NDVI_Awinter2020,SAR_NDVI_Awinter2021,SAR_NDVI_Awinter2022,SAR_NDVI_Awinter2023,SAR_NDVI_Dsummer2018,SAR_NDVI_Dsummer2019,SAR_NDVI_Dsummer2020,SAR_NDVI_Dsummer2021,SAR_NDVI_Dsummer2022,SAR_NDVI_Dsummer2023,SAR_NDVI_Dwinter2018,SAR_NDVI_Dwinter2019,SAR_NDVI_Dwinter2020,SAR_NDVI_Dwinter2021,SAR_NDVI_Dwinter2022,SAR_NDVI_Dwinter2023,DVI_Asummer2018,DVI_Asummer2019,DVI_Asummer2020,DVI_Asummer2021,DVI_Asummer2022,DVI_Asummer2023,DVI_Awinter2018,DVI_Awinter2019,DVI_Awinter2020,DVI_Awinter2021,DVI_Awinter2022,DVI_Awinter2023,DVI_Dsummer2018,DVI_Dsummer2019,DVI_Dsummer2020,DVI_Dsummer2021,DVI_Dsummer2022,DVI_Dsummer2023,DVI_Dwinter2018,DVI_Dwinter2019,DVI_Dwinter2020,DVI_Dwinter2021,DVI_Dwinter2022,DVI_Dwinter2023,SVI_Asummer2018,SVI_Asummer2019,SVI_Asummer2020,SVI_Asummer2021,SVI_Asummer2022,SVI_Asummer2023,SVI_Awinter2018,SVI_Awinter2019,SVI_Awinter2020,SVI_Awinter2021,SVI_Awinter2022,SVI_Awinter2023,SVI_Dsummer2018,SVI_Dsummer2019,SVI_Dsummer2020,SVI_Dsummer2021,SVI_Dsummer2022,SVI_Dsummer2023,SVI_Dwinter2018,SVI_Dwinter2019,SVI_Dwinter2020,SVI_Dwinter2021,SVI_Dwinter2022,SVI_Dwinter2023,RDVI_Asummer2018,RDVI_Asummer2019,RDVI_Asummer2020,RDVI_Asummer2021,RDVI_Asummer2022,RDVI_Asummer2023,RDVI_Awinter2018,RDVI_Awinter2019,RDVI_Awinter2020,RDVI_Awinter2021,RDVI_Awinter2022,RDVI_Awinter2023,RDVI_Dsummer2018,RDVI_Dsummer2019,RDVI_Dsummer2020,RDVI_Dsummer2021,RDVI_Dsummer2022,RDVI_Dsummer2023,RDVI_Dwinter2018,RDVI_Dwinter2019,RDVI_Dwinter2020,RDVI_Dwinter2021,RDVI_Dwinter2022,RDVI_Dwinter2023,NRDVI_Asummer2018,NRDVI_Asummer2019,NRDVI_Asummer2020,NRDVI_Asummer2021,NRDVI_Asummer2022,NRDVI_Asummer2023,NRDVI_Awinter2018,NRDVI_Awinter2019,NRDVI_Awinter2020,NRDVI_Awinter2021,NRDVI_Awinter2022,NRDVI_Awinter2023,NRDVI_Dsummer2018,NRDVI_Dsummer2019,NRDVI_Dsummer2020,NRDVI_Dsummer2021,NRDVI_Dsummer2022,NRDV

In [80]:
species_model_df['species'].value_counts()

species
4     5000
2     5000
11    5000
9     5000
8     5000
7     5000
12    5000
6     4834
10    4765
5     4737
0     4206
1     3939
3     3524
13    3142
Name: count, dtype: int64

### Exporting File

In [81]:
model_df.to_csv(model_df_path, index=False)
species_model_df.to_csv(species_model_df_path, index=False)